In [1]:
# instalar dependencias
%pip install openai langchain langchain_community streamlit
%pip install faiss-cpu
# instalar ChromaDB como retriever 
%pip install chromadb

     ---------------------------------------- 9.8/9.8 MB 13.1 MB/s eta 0:00:00
     ------------------------------------- 731.2/731.2 kB 15.3 MB/s eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 14.9 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 15.5 MB/s eta 0:00:00
     --------------------------------------- 25.8/25.8 MB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 79.1/79.1 kB 4.6 MB/s eta 0:00:00
     -------------------------------------- 207.6/207.6 kB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 6.9/6.9 MB 16.3 MB/s eta 0:00:00
     ---------------------------------------- 88.5/88.5 kB ? eta 0:00:00
     ------------------------------------- 331.9/331.9 kB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00
     ------------------------------------- 509.2/509.2 kB 10.8 MB/s eta 0:00:00
     ------------------------------------- 347.8/3


[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 15.0/15.0 MB 20.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# paso 1:
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv


load_dotenv('api_keys.env')


# Configurar la API Key (puedes establecerla como variable de entorno)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
# bash: export OPENAI_API_KEY="your-openai-api-key"

# Inicializar el modelo GPT-4
llm = ChatOpenAI(model_name="gpt-4")


In [4]:
# Inicializar la Base de Datos Vectorial

import chromadb

# Inicializar la base de datos ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Crear una colección en ChromaDB
collection = chroma_client.get_or_create_collection(name="chatbot_knowledge")

#


In [5]:
# Agregar Datos de Referencia a la Base de Datos

documents = [
    {"id": "1", "text": "La inteligencia artificial es el campo de la informática que estudia cómo crear sistemas capaces de realizar tareas que requieren inteligencia humana."},
    {"id": "2", "text": "GPT-4 es un modelo de lenguaje de OpenAI basado en la arquitectura de transformers."},
    {"id": "3", "text": "LangChain es una biblioteca para desarrollar aplicaciones basadas en modelos de lenguaje."}
]

# Agregar documentos a ChromaDB
for doc in documents:
    collection.add(ids=[doc["id"]], documents=[doc["text"]])


C:\Users\jdmartinev\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:23<00:00, 3.50MiB/s]


In [7]:
# Crear un Retriever con ChromaDB

from langchain_chroma import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Inicializar embeddings con OpenAI
embeddings = OpenAIEmbeddings()

# Crear el VectorStore con ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# Crear el retriever para buscar documentos relevantes
retriever = vectorstore.as_retriever()


In [8]:
# paso 2

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Crear memoria de la conversación
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Integrar memoria en la cadena conversacional

#chat_chain = ConversationalRetrievalChain.from_llm(llm, memory=memory)

# fix con un retriever en chromadb
chat_chain = ConversationalRetrievalChain.from_llm(llm, memory=memory,retriever=retriever)


C:\Users\jdmartinev\AppData\Local\Temp\ipykernel_21296\2690817752.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [9]:
# Paso 3: Construir la Función del Chatbot

def chat_with_gpt(user_input):
    response = chat_chain({"question": user_input})
    return response["answer"]


In [11]:
# Ejecución del Chatbot
%pip install streamlit
%pip install tiktoken



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!streamlit run app.py


In [ ]:
# mejoras propuestas

# Agregar RAG (Retrieval-Augmented Generation) con bases de datos vectoriales y tus propios datos.
# Integrar API externas (ejemplo: Wikipedia, bases de datos de conocimiento).
# Incluir Voice-to-Text para hacer un chatbot por voz.
